In [1]:
from lib.KDTreeEncoding import *

import xgboost as xgb
from lib.XGBHelper import *
from lib.XGBoost_params import *
from lib.score_analysis import *

from lib.logger import logger

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from numpy import load
from glob import glob
import pandas as pd
import pickle as pkl

poverty_dir = '/datasets/cs255-sp22-a00-public/poverty/'
image_dir=poverty_dir+'anon_images/'

In [2]:
import pickle as pkl
pkl_file_rural='data/DumpRural.pk'
pkl_file_urban='data/DumpUrban.pk'
D_rural=pkl.load(open(pkl_file_rural,'rb'))
D_urban=pkl.load(open(pkl_file_urban,'rb'))

for k in D_rural:
    globals()[k]=D_rural[k]
scaling_mean_rural=mean
scaling_std_rural=std
bst_list_rural=[x['bst'] for x in styled_logs[1]['log']]
for k in D_urban:
    globals()[k]=D_urban[k]
scaling_mean_urban=mean
scaling_std_urban=std
bst_list_urban=[x['bst'] for x in styled_logs[1]['log']]


In [3]:
folds=[{'in':'country_test_reduct.csv','out':'results_country.csv'},
      {'in':'random_test_reduct.csv','out':'results.csv'}]

for fold_i in range(len(folds)):
    fold=folds[fold_i]

    test_csv=f'../public_tables/{fold["in"]}'
    test=pd.read_csv(test_csv,index_col=0)
    out=pd.DataFrame()
    out['filename'] = test['filename']
    out['urban']=test['urban']
    out['country']=test['country']
    out['pred_wo_abstention']=0
    out.set_index('filename', inplace=True)
    out['index'] = range(1, len(out) + 1)
    out_urban = out[out['urban']==True]
    out_rural = out[out['urban']==False]
    print(out_urban)
    print(out_rural)
    Enc_data_urban=encoded_dataset(image_dir,out_urban,tree,label_col='pred_wo_abstention')
    Enc_data_rural=encoded_dataset(image_dir,out_rural,tree,label_col='pred_wo_abstention')

    data=to_DMatrix(Enc_data_urban.data)
    Preds=zeros([Enc_data_urban.data.shape[0],len(bst_list_urban)])
    for i in range(len(bst_list_urban)):
        Preds[:,i]=bst_list_urban[i].predict(data,output_margin=True,ntree_limit=bst_list_urban[i].best_ntree_limit)
    Preds=(Preds-scaling_mean_urban)/scaling_std_urban # apply overall score scaling

    _mean=np.mean(Preds,axis=1)
    _std=np.std(Preds,axis=1)

    pred_wo_abstention=(2*(_mean>0))-1
    pred_with_abstention=copy(pred_wo_abstention)
    pred_with_abstention[_std>abs(_mean)]=0

    out_urban['pred_with_abstention'] = pred_with_abstention
    out_urban['pred_wo_abstention'] = pred_wo_abstention
    print(out[out['urban']==True])
    out_urban

    data=to_DMatrix(Enc_data_rural.data)
    Preds=zeros([Enc_data_rural.data.shape[0],len(bst_list_rural)])
    for i in range(len(bst_list_rural)):
        Preds[:,i]=bst_list_rural[i].predict(data,output_margin=True,ntree_limit=bst_list_urban[i].best_ntree_limit)
    Preds=(Preds-scaling_mean_rural)/scaling_std_rural # apply overall score scaling

    _mean=np.mean(Preds,axis=1)
    _std=np.std(Preds,axis=1)

    pred_wo_abstention=(2*(_mean>0))-1
    pred_with_abstention=copy(pred_wo_abstention)
    pred_with_abstention[_std>abs(_mean)]=0

    out_rural['pred_with_abstention'] = pred_with_abstention
    out_rural['pred_wo_abstention'] = pred_wo_abstention
    out_rural

    frames = [out_urban, out_rural]
    result = pd.concat(frames)
    print(result)
    result = result.sort_values(by=['index']).drop(columns=['index'])
    print(result)
    outFile=f'data/{fold["out"]}'
    result.to_csv(outFile)
    print('\n\n'+'-'*60)
    print(outFile)




                urban  country  pred_wo_abstention  index
filename                                                 
image14972.npz   True        1                   0      2
image16964.npz   True        1                   0      3
image6808.npz    True        1                   0      4
image4311.npz    True        1                   0      5
image19407.npz   True        1                   0      6
...               ...      ...                 ...    ...
image6298.npz    True        2                   0   4499
image14371.npz   True        2                   0   4507
image3709.npz    True        2                   0   4513
image11650.npz   True        2                   0   4514
image13495.npz   True        2                   0   4515

[1268 rows x 4 columns]
                urban  country  pred_wo_abstention  index
filename                                                 
image13747.npz  False        1                   0      1
image12845.npz  False        1                 

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


                urban  country  pred_wo_abstention  index
filename                                                 
image14972.npz   True        1                   0      2
image16964.npz   True        1                   0      3
image6808.npz    True        1                   0      4
image4311.npz    True        1                   0      5
image19407.npz   True        1                   0      6
...               ...      ...                 ...    ...
image6298.npz    True        2                   0   4499
image14371.npz   True        2                   0   4507
image3709.npz    True        2                   0   4513
image11650.npz   True        2                   0   4514
image13495.npz   True        2                   0   4515

[1268 rows x 4 columns]


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


                urban  country  pred_wo_abstention  index  \
filename                                                    
image14972.npz   True        1                  -1      2   
image16964.npz   True        1                  -1      3   
image6808.npz    True        1                   1      4   
image4311.npz    True        1                  -1      5   
image19407.npz   True        1                  -1      6   
...               ...      ...                 ...    ...   
image2604.npz   False        2                   1   4509   
image60.npz     False        2                   1   4510   
image19065.npz  False        2                   1   4511   
image10993.npz  False        2                   1   4512   
image6657.npz   False        2                   1   4516   

                pred_with_abstention  
filename                              
image14972.npz                    -1  
image16964.npz                    -1  
image6808.npz                      0  
image4311.n

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [4]:
result

,urban,country,pred_wo_abstention,pred_with_abstention
filename,,,,
image13724.npz,False,6,1,0
image19338.npz,True,6,1,1
image19053.npz,True,6,1,1
image12371.npz,False,6,1,0
image19639.npz,False,6,-1,0
...,...,...,...,...
image9142.npz,False,8,1,1
image7755.npz,False,8,1,1
image13002.npz,True,8,1,1
